In [6]:
import os
import json
import pandas as pd
from pathlib import Path
from datetime import datetime
import nltk
from tqdm import tqdm
import subprocess

In [11]:
nltk.download('punkt', quiet=True)

RAW_CSV = "../../data/raw_speeches.csv"
CACHE_FILE = "../../data/incremental_cache.json"
FED_SCRAPER_DIR = "../../Fed-Scraper-main/fed_scraper"
FED_SCRAPER_MAIN_DIR = "../../Fed-Scraper-main/"
DOCUMENTS_DIR = os.path.join(FED_SCRAPER_MAIN_DIR, "data", "documents_by_type")

CURRENT_DIR = Path.cwd() 
FED_SCRAPER_ROOT_ABS = (CURRENT_DIR / FED_SCRAPER_DIR).resolve()
print(FED_SCRAPER_ROOT_ABS)
FED_SCRAPER_MAIN_DIR_ROOT = (CURRENT_DIR / FED_SCRAPER_MAIN_DIR).resolve()
print(FED_SCRAPER_MAIN_DIR_ROOT)
RAW_CSV_ABS = (CURRENT_DIR / RAW_CSV).resolve()
CACHE_FILE_ABS = (CURRENT_DIR / CACHE_FILE).resolve()

INCLUDED_KINDS = {
    "fomc", "minutes", "press_conferences", "speeches",
    "transcripts", "projections", "beige_book", "redbooks", "teal_book"
}

class FedSpeechFetcher:
    def __init__(self):
        if not os.path.exists("data"):
            os.makedirs("data")
    
    def run_spiders(self):
        """
        Run all spiders (full scraping on first execution)
        """
        spiders = [
            "beige_book_archive",
            "beige_book_current",
            "fomc_calendar",
            "historical_materials"
        ]

        for spider in spiders:
            #subprocess.run(
            #    ["scrapy", "crawl", spider],
            #    cwd=FED_SCRAPER_DIR,
            #    check=True)
            # Capturing output is the key change!
            scrapy_cmd = str(FED_SCRAPER_ROOT_ABS)
            print(scrapy_cmd)
            result = subprocess.run(
                ["scrapy", "crawl", spider],
                cwd=scrapy_cmd,
                capture_output=True,  # Capture stdout and stderr
                text=True,            # Decode output as text
                check=False           # Temporarily set to False so it doesn't crash Python
            )
            
            # If the process returned an error code, print the full traceback from Scrapy
            if result.returncode != 0:
                print(f"ERROR: Scrapy spider '{spider}' failed with exit status {result.returncode}")
                print("\n--- Scrapy STDOUT ---")
                print(result.stdout)
                print("\n--- Scrapy STDERR (Actual Error Message) ---")
                print(result.stderr)
                
                # Now raise the error manually so the calling function knows it failed
                result.check_returncode()

    def load_fed_scraper_outputs(self):
        frames = []
        DOCUMENTS_DIR_ABS = FED_SCRAPER_MAIN_DIR_ROOT / "data" / "documents_by_type"
        print("Documents dir exists:", DOCUMENTS_DIR_ABS ,DOCUMENTS_DIR_ABS.exists())
        if not DOCUMENTS_DIR_ABS.exists():
            raise FileNotFoundError(
                f"Fed-Scraper output directory not found: {DOCUMENTS_DIR_ABS}\n"
                "Make sure spiders completed successfully."
            )
            
        for fname in DOCUMENTS_DIR_ABS.iterdir():
            if not fname.name.endswith(".csv"):
                continue

            kind = fname.stem

            if kind not in INCLUDED_KINDS:
                continue

            df = pd.read_csv(fname)
            df["document_kind"] = kind
            frames.append(df)

        if not frames:
            return pd.DataFrame()

        return pd.concat(frames, ignore_index=True)
    
    def load_cache(self):
        if not os.path.exists(CACHE_FILE_ABS):
            return {}
        with open(CACHE_FILE_ABS, "r") as f:
            return json.load(f)
        
    def save_cache(self, cache):
        Path(CACHE_FILE_ABS).parent.mkdir(parents=True, exist_ok=True) 
        with open(CACHE_FILE_ABS, "w") as f:
            json.dump(cache, f, indent=2)
            
    def append_new_speeches(self, df):
        """
        write new rows to data/raw_speeches.csv
        """
        cache = self.load_cache()

        new_rows = []

        for _, row in df.iterrows():
            uid = f"{row['document_kind']}_{row['url']}"

            release = row.get("release_date", None)
            if isinstance(release, float):
                release = None

            if uid not in cache:
                new_rows.append(row)
                cache[uid] = release
            else:
                if release and release != cache[uid]:
                    new_rows.append(row)
                    cache[uid] = release

        # append to disk
        if new_rows:
            new_df = pd.DataFrame(new_rows)
            Path(RAW_CSV_ABS).parent.mkdir(parents=True, exist_ok=True) # Ensure parent directory exists

            if not os.path.exists(RAW_CSV_ABS):
                new_df.to_csv(RAW_CSV_ABS, index=False)
            else:
                old = pd.read_csv(RAW_CSV_ABS)
                full = pd.concat([old, new_df], ignore_index=True)
                full.to_csv(RAW_CSV_ABS, index=False)

        self.save_cache(cache)

    def run_full(self):
        self.run_spiders()
        df = self.load_fed_scraper_outputs()
        self.append_new_speeches(df)
        return df

    def run_incremental(self):
        df = self.load_fed_scraper_outputs()
        self.append_new_speeches(df)
        return df
    
if __name__ == "__main__":
    fetcher = FedSpeechFetcher()

if not os.path.exists(RAW_CSV):
    print("Performing full scrape...")
    fetcher.run_full()
else:
    print("Performing incremental scrape...")
    fetcher.run_incremental()

C:\Users\windows\OneDrive - VietNam National University - HCM INTERNATIONAL UNIVERSITY\Desktop\Programming\Project\Price-Prediction-AI\Fed-Scraper-main\fed_scraper
C:\Users\windows\OneDrive - VietNam National University - HCM INTERNATIONAL UNIVERSITY\Desktop\Programming\Project\Price-Prediction-AI\Fed-Scraper-main
Performing incremental scrape...
Documents dir exists: C:\Users\windows\OneDrive - VietNam National University - HCM INTERNATIONAL UNIVERSITY\Desktop\Programming\Project\Price-Prediction-AI\Fed-Scraper-main\data\documents_by_type True
